In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
client = makeEfdClient()
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

In [ ]:
start = Time("2024-04-02T07:40:00", scale='utc')
end = Time("2024-04-02T07:53:00", scale='utc')

In [ ]:
hexapod = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.positionStatus",
    columns="*",
    begin=start, end=end)

inPos = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.logevent_inPosition",
    columns="*",
    begin=start, end=end)
inPos = inPos[inPos['inPosition']]
corrStart = getEfdData(
    client=client,
    topic="lsst.sal.ATAOS.logevent_hexapodCorrectionStarted",
    columns="*",
    begin=start, end=end)

corrComplete = getEfdData(
    client=client,
    topic="lsst.sal.ATAOS.logevent_hexapodCorrectionCompleted",
    columns="*",
    begin=start, end=end)


In [ ]:
inPos['private_efdStamp'].values

In [ ]:
len(corrStart.index)

In [ ]:
%matplotlib inline
hexapod['reportedPosition2'].plot(marker='x')
hexapod['setpointPosition2'].plot(marker='x')
for pos in inPos['private_efdStamp'].values:
    plt.axvline(Time(pos, format='unix_tai').utc.isot, color='black', ls='--', label='InPosition')
for cstrt in corrStart['private_efdStamp'].values:
    plt.axvline(Time(cstrt, format='unix_tai').utc.isot, color='green', ls='--', label='CorrectionStart')
for cplt in corrComplete['private_efdStamp'].values:
    plt.axvline(Time(cplt, format='unix_tai').utc.isot, color='red', ls='-.', label='CorrectionComplete')
plotStart = Time("2024-04-02T07:42:05", scale='utc').isot
plotEnd = Time("2024-04-02T07:42:59", scale='utc').isot
plt.xlim(plotStart, plotEnd)
#plt.legend()

In [ ]:
hexapod['reportedPosition2'].plot(marker='x')

In [ ]:
expId = 2024032100122
mData = butler.get('raw.metadata', exposure=expId, detector=0)
expStart = Time(mData['DATE-BEG'], scale='tai').utc
start = expStart - TimeDelta(10.0, format='sec')
expEnd = Time(mData['DATE-END'], scale='tai').utc
end = expEnd + TimeDelta(0.5, format='sec')
hexapod = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.positionStatus",
    columns="*",
    begin=start, end=end)

inPos = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.logevent_inPosition",
    columns="*",
    begin=start, end=end)
inPos = inPos[inPos['inPosition']]
corrStart = getEfdData(
    client=client,
    topic="lsst.sal.ATAOS.logevent_hexapodCorrectionStarted",
    columns="*",
    begin=start, end=end)

corrComplete = getEfdData(
    client=client,
    topic="lsst.sal.ATAOS.logevent_hexapodCorrectionCompleted",
    columns="*",
    begin=start, end=end)
hexapod['reportedPosition2'].plot(marker='x')
hexapod['setpointPosition2'].plot(marker='+')
for pos in inPos.index:
    plt.axvline(pos.isoformat(), color='black', ls='--', label='HexInPosition')
for cstrt in corrStart.index:
    plt.axvline(cstrt.isoformat(), color='green', ls='--', label='CorrectionStart')
for cplt in corrComplete.index:
    plt.axvline(cplt.isoformat(), color='red', ls='-.', label='CorrectionComplete')
plt.axvline(expStart.isot, color='magenta', ls='--', label='ExposureStart')
plt.axvline(expEnd.isot, color='cyan', ls='--', label='ExposureEnd')
plt.legend()

In [ ]:
inPos.index[0].isoformat()

In [ ]:
expIds = range(2024032100121, 2024032100130)
expIds = range(2024032100242, 2024032100251)
values = []
for expId in expIds:
    mData = butler.get('raw.metadata', exposure=expId, detector=0)
    start = Time(mData['DATE-BEG'], scale='tai').utc
    end = Time(mData['DATE-END'], scale='tai').utc
    hexapod = getEfdData(
        client=client,
        topic="lsst.sal.ATHexapod.positionStatus",
        columns="*",
        begin=start, end=end)
    value = hexapod['reportedPosition2'].values.mean()
    values.append(value)
    print(expId, value)
    

In [ ]:
plt.plot(expIds, values, marker='x')

In [ ]:
hexapod['reportedPosition2'].plot()

In [ ]:
expIds = [2024032100121, 2024032100129]
mData = butler.get('raw.metadata', exposure=expIds[0], detector=0)
start = Time(mData['DATE-BEG'], scale='tai').utc - TimeDelta(8.0, format='sec')
mData = butler.get('raw.metadata', exposure=expIds[1], detector=0)
end = Time(mData['DATE-END'], scale='tai').utc + TimeDelta(2.0, format='sec')
hexapod = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.positionStatus",
    columns="*",
    begin=start, end=end)
inPos = getEfdData(
    client=client,
    topic="lsst.sal.ATHexapod.logevent_inPosition",
    columns="*",
    begin=start, end=end)
inPos = inPos[inPos['inPosition']]

expIds = range(2024032100121, 2024032100130)
starts = []
ends = []
for expId in expIds:
    mData = butler.get('raw.metadata', exposure=expId, detector=0)
    start = Time(mData['DATE-BEG'], scale='tai').utc
    end = Time(mData['DATE-END'], scale='tai').utc
    starts.append(start)
    ends.append(end)

fig = plt.figure(figsize=(15,5))
plt.title(f"Hexapod Z position during focus sweep -{expIds[0]}, {expIds[-1]}", fontsize=18)
hexapod['reportedPosition2'].plot(marker='x')
plt.axvline(starts[0].isot, color='green', ls='--', label='ExposureStart')
plt.axvline(ends[0].isot, color='red', ls='--', label='ExposureEnd')
plt.axvline(inPos.index[0].isoformat(), color='black', ls='--', label='HexInPosition')
for pos in inPos.index:
    plt.axvline(pos.isoformat(), color='black', ls='--', label='')
for start in starts:
    plt.axvline(start.isot, color='green', ls='--', label='')
for end in ends:
    plt.axvline(end.isot, color='red', ls='--', label='')
plt.ylabel("Hexapod-Z (mm)")
plt.ylim(-1.25, -1.0)
plt.legend(loc='upper left')
plt.savefig("/home/c/cslage/u/AuxTel/fwhm/Focus_Sweep_Hexapod_02Apr24.pdf")

In [ ]:
for i, value in enumerate(hexapod['reportedPosition2'].values):
    print(i, value)

In [ ]:
hexapod['reportedPosition2'].values[10] - hexapod['reportedPosition2'].values[32]